In [1]:
import QuantLib as ql

precio_activo = 440 #Spot (S)
precio_ejercicio = 450 #Strike (K)
volatilidad = 0.3394  #Volatilidad Implicita anualizada (sigma)
tasa_interes = 0.0013 #tasa libre de riesgo anualizada (r)
tasa_dividendos =  0.075 #tasa de dividendos anualizada (q)

#Expiracion el 16 de octubre de 2020
fecha_expiracion = ql.Date(6, 8, 2021)

#Valuacion el 16 de octubre de 2020
fecha_valuacion = ql.Date(6, 8, 2020)
ql.Settings.instance().evaluationDate = fecha_valuacion

#Calendario y convencion de daycount
day_count = ql.Actual365Fixed()
calendario = ql.UnitedStates()


tipo_opcion = ql.Option.Call #Tipo de opcion (CALL o PUT)
payoff = ql.PlainVanillaPayoff(tipo_opcion, precio_ejercicio)
ejercicio_europeo = ql.EuropeanExercise(fecha_expiracion)


opcion_europea = ql.VanillaOption(payoff, ejercicio_europeo)

objeto_spot = ql.QuoteHandle(ql.SimpleQuote(precio_activo))

objeto_tasa_interes = ql.YieldTermStructureHandle(ql.FlatForward(fecha_valuacion, 
                                                                tasa_interes, 
                                                                day_count))

tasa_recup = objeto_tasa_interes.forwardRate(fecha_valuacion, fecha_expiracion, day_count, ql.Continuous).rate()

objeto_tasa_dividendos = ql.YieldTermStructureHandle(ql.FlatForward(fecha_valuacion, 
                                                      tasa_dividendos, 
                                                      day_count))
div_recup = objeto_tasa_dividendos.forwardRate(fecha_valuacion, fecha_expiracion, day_count, ql.Continuous).rate()

objeto_volatilidad = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(fecha_valuacion, 
                                                                 calendario, 
                                                                 volatilidad, 
                                                             day_count))
vol_recup = objeto_volatilidad.blackVol(fecha_expiracion, precio_ejercicio)

proceso_BSM = ql.BlackScholesMertonProcess(objeto_spot, 
                                           objeto_tasa_dividendos, 
                                           objeto_tasa_interes, 
                                           objeto_volatilidad)



## Black Sholes QuantLib


In [2]:
modelo_BS = ql.AnalyticEuropeanEngine(proceso_BSM)

opcion_europea.setPricingEngine(modelo_BS)

precio_opcion_BS = opcion_europea.NPV()

print("El precio teorico usando el modelo de BS es: ", precio_opcion_BS)

El precio teorico usando el modelo de BS es:  39.46009202020682


## FD QuantLib


In [3]:
tGrid = 100
xGrid = 100
dampingSteps = 0
esquema_df = ql.FdmSchemeDesc.MethodOfLines()

modelo_DF = ql.FdBlackScholesVanillaEngine(proceso_BSM, 
                                           tGrid, 
                                           xGrid,
                                           dampingSteps,
                                           esquema_df)

opcion_europea.setPricingEngine(modelo_DF)

precio_opcion_DF_ql = opcion_europea.NPV()

print("El precio teorico usando el modelo de Dif. Finitas (ql) es: ", precio_opcion_DF_ql)

El precio teorico usando el modelo de Dif. Finitas (ql) es:  39.47637917085886


## Modelos FD Manu 

In [4]:
from opcion_europea_fd import opcion_europea_fd as fd


T = 1
tipo = "C"

M = 160



In [5]:
precio_opcion_DF_manu = fd(tipo,precio_activo,precio_ejercicio,T,tasa_interes,volatilidad,tasa_dividendos, M)


print("El precio teorico usando el modelo de Dif. Finitas (manu) es: ", precio_opcion_DF_manu)

El precio teorico usando el modelo de Dif. Finitas (manu) es:  39.455269474163075


In [16]:
import QuantLib as ql

T = 63
fecha_valuacion = ql.Date(6, 8, 2022)
ql.Settings.instance().evaluationDate = fecha_valuacion
period = str(T)+"D"
calendario = ql.UnitedStates()
fecha_expiracion = calendario.advance(fecha_valuacion,ql.Period(period))
day_count = ql.Actual365Fixed()

#print(fecha_expiracion)

aaa = ql.Date(6, 8, 2022) + 8

print(aaa)


August 14th, 2022
